## Computational Macroeconomics Assignment 5
Paula Beck

handed in: 29.01.2022

In [428]:
# some preparation

# import packages
from pathlib import Path
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols


# set path
wd = Path()

### Step 1: download data and save
downloaded on 17.01.2022

### Step 2: load data

In [429]:
# load consumer expectation data

# instead of changing the excel file manually, I load the specific data directly in python by loading the explicite sheet and skipping the first three rows. Also, I only load the first three columns as those are the one we need.
exp = pd.read_excel(wd/'data'/'FRBNY-SCE-Data.xlsx', sheet_name = 'Inflation expectations', skiprows=3, usecols=[0,1,2])

exp.rename(columns={exp.columns[0]:'date'}, inplace=True)

print(exp.head())

     date  Median one-year ahead expected inflation rate  \
0  201306                                       3.090884   
1  201307                                       3.162522   
2  201308                                       3.395071   
3  201309                                       3.367290   
4  201310                                       3.174733   

   Median three-year ahead expected inflation rate  
0                                         3.416846  
1                                         3.310545  
2                                         3.799491  
3                                         3.546918  
4                                         3.196597  


In [430]:
# load inflation data

inf = pd.read_csv(wd/'data'/'CPALTT01USM659N.csv', sep = ',', decimal ='.', names=['date', 'inflation'], header=0)

print(inf.head())

         date  inflation
0  1960-01-01   1.034483
1  1960-02-01   1.730104
2  1960-03-01   1.730104
3  1960-04-01   1.724138
4  1960-05-01   1.724138


### Step 3: convert to datetime

In [431]:
# first look what data types they have
print('---data types of inflation data------------')
print(inf.dtypes)
print('---data types of expectation data----------')
print(exp.dtypes)

---data types of inflation data------------
date          object
inflation    float64
dtype: object
---data types of expectation data----------
date                                                 int64
Median one-year ahead expected inflation rate      float64
Median three-year ahead expected inflation rate    float64
dtype: object


In [432]:
# convert date column of inflation dataframe to datetime
inf.date  = pd.to_datetime(inf.date)
# set frequency to monthly
inf.date = pd.PeriodIndex(inf.date, freq='m')

# convert the date column of the expectation dataframe to a string and add '01', then convert to datetime
exp.date = exp.date.astype('str') + '01'
exp.date = pd.to_datetime(exp.date, format = '%Y%m%d')
# set frequency to monthly
exp.date = pd.PeriodIndex(exp.date, freq='m')


In [433]:
# set date as index
inf = inf.set_index('date')
exp = exp.set_index('date')

### Step 4: rename columns

In [434]:
exp.rename(columns={exp.columns[0]:'med1y',exp.columns[1]:'med3y'}, inplace=True)

### Step 5: merge dataframes

In [435]:
# merge expectation data to inflation dataframe based on the index
# inner merge such that only those rows remain that are not NA for both data sources
df = inf.merge(exp, on='date', how='inner')

### Step 6: calculate forecast errors and forecast revision

In [436]:
####### check again !!!!!

# set all to point t

df['ferror'] = df.inflation.shift(-12) - df.med1y 
df['frevision'] = df.med1y - df.med3y.shift(24) 
df

,inflation,med1y,med3y,ferror,frevision
date,,,,,
2013-06,1.754417,3.090884,3.416846,-1.018543,NaN
2013-07,1.960682,3.162522,3.310545,-1.170193,NaN
2013-08,1.518368,3.395071,3.799491,-1.695460,NaN
2013-09,1.184925,3.367290,3.546918,-1.709371,NaN
2013-10,0.963613,3.174733,3.196597,-1.510392,NaN
...,...,...,...,...,...
2021-06,5.391451,4.801745,3.546918,NaN,2.139308
2021-07,5.365475,4.842451,3.711669,NaN,2.263085
2021-08,5.251272,5.179827,3.999662,NaN,2.678395


### Step 7: drop all columns except from forecast errors and forecast revision

In [437]:
df = df[['ferror', 'frevision']]

### Step 8: remove NAN values

In [438]:
df = df.dropna()
df.head()

,ferror,frevision
date,,
2015-06,-2.002674,-0.416846
2015-07,-2.128218,-0.355189
2015-08,-1.730700,-1.005916
2015-09,-1.270875,-0.812259
2015-10,-1.187338,-0.373271


### Step 9: estimate equation (1)

- equation: ferror = constant + frevision + error
- set cov_type = 'HAC'
- set cov_kwds = {'maxlags':3}

In [441]:
model = ols(formula='ferror ~ frevision', data=df)
fit = model.fit(cov_type = 'HAC', cov_kwds = {'maxlags':3})
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ferror   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                   0.04515
Date:                Mon, 17 Jan 2022   Prob (F-statistic):              0.832
Time:                        18:00:21   Log-Likelihood:                -103.28
No. Observations:                  65   AIC:                             210.6
Df Residuals:                      63   BIC:                             214.9
Df Model:                           1                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.5279      0.319     -1.656      0.098      -1.153       0.097
frevision     -0.1007      0.474     -0.212      0.832      -1.030       0.828
==============================================================================
Omnibus:                       24.573   Durbin-Watson:                   0.114
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               36.159
Skew:                           1.484   Prob(JB):                     1.41e-08
Kurtosis:                       5.130   Cond. No.                         3.15
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 3 lags and without small sample correction
"""

### Step 10: Interpretation

- correlation
- Rational Expectation: $\beta$ should be not significantlly different from 0


- small sample size